## Atomman and Mat3ra integration example

Installing both packages and related dependencies in JupyterLite/pyodide,
Loading material from environment or Standata
Using Atomman to create a vacancy defect in a material.
Converting back to Mat3ra material and visualizing it.

In [ ]:
# import Mat3ra related packages
import sys

if sys.platform == "emscripten":
    import micropip

    # Install and import atomman related packages
    micropip.uninstall("packaging")
    await micropip.install("packaging==24.1")
    await micropip.install("https://files.mat3ra.com/uploads/pyyaml-6.0.2-cp311-cp311-emscripten_3_1_45_wasm32.whl")
    await micropip.install("https://files.mat3ra.com/uploads/pymongo-4.15.1-cp311-cp311-emscripten_3_1_45_wasm32.whl")
    await micropip.install("https://files.mat3ra.com/uploads/atomman-1.4.11-cp311-cp311-emscripten_3_1_45_wasm32.whl",keep_going=True)

    # Install and import mat3ra related packages
    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    await micropip.install('mat3ra-standata', deps=False)
    from mat3ra.utils.jupyterlite.packages import install_packages
    await install_packages("")

In [ ]:
# Load material from inputs
from utils.jupyterlite import get_materials
from mat3ra.made.tools.convert import to_poscar
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

materials = get_materials(globals())
material = materials[0] or Material.create(Materials.get_by_name_first_match("Si"))

poscar = to_poscar(material)

In [ ]:
import atomman as am
# Load poscar and pass in prop dictionary
system = am.load('poscar', poscar)
print(system)
system = system.supersize(2,2,1)
system.atoms_df()

In [ ]:
# Add a vacancy defect
defected_system = am.defect.vacancy(system, ptd_id=0)

In [ ]:
# Convert back to Mat3ra material
from mat3ra.made.tools.convert import from_poscar

defected_system_poscar = defected_system.dump("poscar")
defected_material = Material.create(from_poscar(defected_system_poscar))

In [ ]:
from utils.visualize import visualize_materials

visualize_materials(defected_material)